In [1]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

In [2]:
import pandas as pd
import numpy as np
import re
from src import fe as fe

## Importamos información

In [3]:
path = 's3://inai-summerofdata/clean/inai.parquet'
df = pd.read_parquet(path, engine='pyarrow')
pd.options.display.max_columns = None

In [4]:
df.head()

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
0,1857500001912,2012-01-01 05:04:07,pemex exploracion y produccion,terminada,electronica,informacion publica,copia de los documentos y los datos con que cu...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 02...
1,1857500002012,2012-01-01 05:13:16,pemex exploracion y produccion,terminada,electronica,informacion publica,del atg para precisar informacion y datos usad...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 01...
2,0001800000712,2012-01-01 09:46:39,secretaria de energia,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,inexistencia de la informacion solicitada,el asunto se turno a la direccion general de r...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,energia,zip,2.0,None
3,0000600003712,2012-01-01 10:11:41,secretaria de hacienda y credito publico,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,la informacion esta disponible publicamente,direcci&oacute;n de internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,hacienda y credito publico,pdf,1.0,inidos secretaria de hacienda y credito public...
4,0001500002812,2012-01-01 10:32:15,secretaria de desarrollo agrario territorial y...,terminada,electronica,informacion publica,agradecere se me proporcione en archivo magnet...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,mexico,distrito federal,miguel hidalgo,11230,reforma agraria,zip,4.0,None


**Importamos stopwords**

In [5]:
stopwords = pd.read_csv('/Users/diegovillalizarraga/github/score-respuesta-inai/safari_transparencia/notebooks/stopwordsadicionalesinai.txt', sep=",", header=0)

In [6]:
stopwords.head()

,Palabras para quitar de Wordclouds
0,65
1,017skc002
2,20oa2008u
3,25s
4,60m


In [7]:
stopwords.tail()

,Palabras para quitar de Wordclouds
882,web
883,word
884,xal
885,youtube
886,zetas


## Eliminamos stopwords de la columna descripcionsolicitud

Primero metemos todas las stopwords en una lista.

In [8]:
banned = []
for i in range(len(stopwords)):
    banned.append(stopwords['Palabras para quitar de Wordclouds'].iloc[i])

Quitamos todas las stopwords de la columna descripcionsolicitud.

In [9]:
f = lambda x: ' '.join([item for item in x.split() if item not in banned])
df['descripcionsolicitud'] = df['descripcionsolicitud'].apply(f)

In [10]:
df.head()

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto
0,1857500001912,2012-01-01 05:04:07,pemex exploracion y produccion,terminada,electronica,informacion publica,copia de los documentos y los con que cuenta a...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 02...
1,1857500002012,2012-01-01 05:13:16,pemex exploracion y produccion,terminada,electronica,informacion publica,del informacion y usados en la de flujo de efe...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 01...
2,0001800000712,2012-01-01 09:46:39,secretaria de energia,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,inexistencia de la informacion solicitada,el asunto se turno a la direccion general de r...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,energia,zip,2.0,None
3,0000600003712,2012-01-01 10:11:41,secretaria de hacienda y credito publico,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,la informacion esta disponible publicamente,direcci&oacute;n de internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,hacienda y credito publico,pdf,1.0,inidos secretaria de hacienda y credito public...
4,0001500002812,2012-01-01 10:32:15,secretaria de desarrollo agrario territorial y...,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,mexico,distrito federal,miguel hidalgo,11230,reforma agraria,zip,4.0,None


## Creamos columna respuestareal

Primero duplicamos la columna respuesta.

In [11]:
df['respuestareal'] = df['respuesta']

In [12]:
df.head()

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto,respuestareal
0,1857500001912,2012-01-01 05:04:07,pemex exploracion y produccion,terminada,electronica,informacion publica,copia de los documentos y los con que cuenta a...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 02...,entrega de informacion en medio electronico
1,1857500002012,2012-01-01 05:13:16,pemex exploracion y produccion,terminada,electronica,informacion publica,del informacion y usados en la de flujo de efe...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 01...,entrega de informacion en medio electronico
2,0001800000712,2012-01-01 09:46:39,secretaria de energia,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,inexistencia de la informacion solicitada,el asunto se turno a la direccion general de r...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,energia,zip,2.0,None,inexistencia de la informacion solicitada
3,0000600003712,2012-01-01 10:11:41,secretaria de hacienda y credito publico,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,la informacion esta disponible publicamente,direcci&oacute;n de internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,hacienda y credito publico,pdf,1.0,inidos secretaria de hacienda y credito public...,la informacion esta disponible publicamente
4,0001500002812,2012-01-01 10:32:15,secretaria de desarrollo agrario territorial y...,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,mexico,distrito federal,miguel hidalgo,11230,reforma agraria,zip,4.0,None,entrega de informacion en medio electronico


In [13]:
df['respuestareal'].unique()

array(['entrega de informacion en medio electronico',
       'inexistencia de la informacion solicitada',
       'la informacion esta disponible publicamente',
       'no es de competencia de la unidad de enlace',
       'requerimiento de informacion adicional', 'notificacion de envio',
       'no se dara tramite a la solicitud',
       'negativa por ser reservada o confidencial',
       'la solicitud no corresponde al marco de la ley',
       'notificacion de disponibilidad de informacion',
       'notificacion lugar y fecha de entrega',
       'informacion parcialmente reservada o confidencial',
       'respuesta del solicitante a la notificacion de entrega de informacion con costo',
       'sin respuesta',
       'respuesta del solicitante a la notificacion de entrega de informacion sin costo',
       'notificacion de prorroga', 'notificacion de pago',
       'respuesta a solicitud de informacion adicional',
       'notificacion de cambio de tipo de solicitud'], dtype=object)

In [14]:
df['tipo_archivo_respuesta'].unique()

array(['pdf', 'zip', 'doc', 'None', 'PDF', 'xls', 'ZIP', 'DOC', 'XLS',
       'docx', 'Pdf', 'txt', 'rar', 'msg', 'xlsx'], dtype=object)

### Ahora importamos las frases que consideraremos como un rechazo si estas se encuentran en el texto de los archivos pdf

In [15]:
rechazo = pd.read_csv('/Users/diegovillalizarraga/github/score-respuesta-inai/safari_transparencia/notebooks/Rechazos en adjuntos - palabras y frases.txt', sep=",", header=0)

In [16]:
rechazo.head()

,Rechazos en adjuntos
0,Inexistente
1,Reservada
2,Reservada parcialmente
3,Después de una búsqueda exhaustiva no se encon...
4,Información confidencial


Pasamos todas las frases a lowercase.

In [17]:
filtro = rechazo.dtypes == np.object
objects = rechazo.dtypes[filtro]
StringColumns = list(objects.index)

In [18]:
for col in StringColumns:
    rechazo[col] = rechazo[col].str.lower()

In [19]:
rechazo.head()

,Rechazos en adjuntos
0,inexistente
1,reservada
2,reservada parcialmente
3,después de una búsqueda exhaustiva no se encon...
4,información confidencial


Eliminamos acentos.

In [20]:
for col in StringColumns:
    rechazo[col] = rechazo[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [21]:
rechazo.head()

,Rechazos en adjuntos
0,inexistente
1,reservada
2,reservada parcialmente
3,despues de una busqueda exhaustiva no se encon...
4,informacion confidencial


Eliminamos espacios al inicio y al final.

In [22]:
for col in StringColumns:
    rechazo[col] = rechazo[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

In [23]:
rechazo.head()

,Rechazos en adjuntos
0,inexistente
1,reservada
2,reservada parcialmente
3,despues de una busqueda exhaustiva no se encon...
4,informacion confidencial


Eliminamos espacios dobles (o mas).

In [24]:
for col in StringColumns:
    rechazo[col] = rechazo[col].apply(lambda x: re.sub(' +', ' ', x) if isinstance(x, str) else x)

In [25]:
rechazo.head()

,Rechazos en adjuntos
0,inexistente
1,reservada
2,reservada parcialmente
3,despues de una busqueda exhaustiva no se encon...
4,informacion confidencial


Creamos lista de frases que implican un rechazo.

In [26]:
validacion_texto = []
for i in range(len(rechazo)):
    validacion_texto.append(rechazo['Rechazos en adjuntos'].iloc[i])

In [27]:
validacion_texto

['inexistente',
 'reservada',
 'reservada parcialmente',
 'despues de una busqueda exhaustiva no se encontro informacion',
 'informacion confidencial',
 'no somos competentes',
 'confidencial',
 'inexistencia',
 'despues de una busqueda exhaustiva no se encontro informacion',
 'se informa que los documentos solicitados se consideran clasificados como reservados por seguridad nacional',
 'no es el organismo facultado para dar contestacion a dicha solicitud',
 'en razon de lo expuesto por las areas competentes se sugiere turnar su solicitud de informacion directamente a',
 'se declara la inexistencia de la informacion solicitada',
 'la informacion genera costos',
 'no se cuenta con dato',
 'puede consultar la informacion en la siguiente direccion',
 'investigacion en curso',
 'se tienen que hacer versiones publicas',
 'no podemos generar informacion ad hoc',
 'documento inmerso en negociaciones internacionales con otros paises',
 'la dependencia no es el sujeto obligado que genera la inf

Creamos una columna nueva con el flag que nos indique si la columna texto_respuesta_adjunto contiene frases de rechazo.

In [28]:
df['flag_rechazo'] = df['texto_respuesta_adjunto'].apply(lambda x: any([k in x for k in validacion_texto]))

In [29]:
df.head()

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto,respuestareal,flag_rechazo
0,1857500001912,2012-01-01 05:04:07,pemex exploracion y produccion,terminada,electronica,informacion publica,copia de los documentos y los con que cuenta a...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 02...,entrega de informacion en medio electronico,False
1,1857500002012,2012-01-01 05:13:16,pemex exploracion y produccion,terminada,electronica,informacion publica,del informacion y usados en la de flujo de efe...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 01...,entrega de informacion en medio electronico,False
2,0001800000712,2012-01-01 09:46:39,secretaria de energia,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,inexistencia de la informacion solicitada,el asunto se turno a la direccion general de r...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,energia,zip,2.0,None,inexistencia de la informacion solicitada,False
3,0000600003712,2012-01-01 10:11:41,secretaria de hacienda y credito publico,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,la informacion esta disponible publicamente,direcci&oacute;n de internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,hacienda y credito publico,pdf,1.0,inidos secretaria de hacienda y credito public...,la informacion esta disponible publicamente,False
4,0001500002812,2012-01-01 10:32:15,secretaria de desarrollo agrario territorial y...,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,mexico,distrito federal,miguel hidalgo,11230,reforma agraria,zip,4.0,None,entrega de informacion en medio electronico,False


In [30]:
filtro = df['flag_rechazo'] == True
df_subset = df[filtro]

In [31]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71146 entries, 22 to 1554538
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   folio                        71146 non-null  object        
 1   fechasolicitud               71146 non-null  datetime64[ns]
 2   dependencia                  71146 non-null  object        
 3   estatus                      71146 non-null  object        
 4   medioentrada                 71146 non-null  object        
 5   tiposolicitud                71146 non-null  object        
 6   descripcionsolicitud         71146 non-null  object        
 7   otrosdatos                   71146 non-null  object        
 8   archivoadjuntosolicitud      71146 non-null  object        
 9   medioentrega                 71146 non-null  object        
 10  fechalimite                  68337 non-null  datetime64[ns]
 11  respuesta                    71146 non

In [32]:
df_subset.head()

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto,respuestareal,flag_rechazo
22,0001200005312,2012-01-01 14:15:43,secretaria de salud,terminada,electronica,informacion publica,con fundamento en previsto en los articulos 4 ...,por favor no referenciar a compranet; la prese...,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,favor de ver el archivo anexo.,https://www.infomex.org.mx/gobiernofederal/mod...,2012-01-17,mexico,mexico,naucalpan de juarez,53840,salud,pdf,1.0,20t secre 16 ene 2012 comision coordinadora de...,entrega de informacion en medio electronico,True
23,0001200005412,2012-01-01 14:18:31,secretaria de salud,terminada,electronica,informacion publica,con fundamento en previsto en los articulos 4 ...,por favor no referenciar a compranet; la prese...,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,favor de ver el archivo anexo.,https://www.infomex.org.mx/gobiernofederal/mod...,2012-01-16,mexico,mexico,naucalpan de juarez,53840,salud,pdf,1.0,39 idos comision coordinadora de institutos na...,entrega de informacion en medio electronico,True
24,0001200005512,2012-01-01 14:21:28,secretaria de salud,terminada,electronica,informacion publica,con fundamento en previsto en los articulos 4 ...,por favor no referenciar a compranet; la prese...,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,favor de ver el archivo anexo.,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,mexico,naucalpan de juarez,53840,salud,pdf,1.0,0492 comision coordinadora de institutos nacio...,entrega de informacion en medio electronico,True
25,0001200005612,2012-01-01 14:24:43,secretaria de salud,terminada,electronica,informacion publica,con fundamento en previsto en los articulos 4 ...,por favor no referenciar a compranet; la prese...,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,favor de ver el archivo anexo.,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,mexico,naucalpan de juarez,53840,salud,pdf,1.0,4h comision coordinadora de institutos naciona...,entrega de informacion en medio electronico,True
26,0001200005712,2012-01-01 14:28:28,secretaria de salud,terminada,electronica,informacion publica,con fundamento en previsto en los articulos 4 ...,por favor no referenciar a compranet; la prese...,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,favor de ver el archivo anexo.,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,mexico,naucalpan de juarez,53840,salud,pdf,1.0,$89 comision coordinadora de institutos nacion...,entrega de informacion en medio electronico,True


In [33]:
df_subset['texto_respuesta_adjunto'].loc[22]

'20t secre 16 ene 2012 comision coordinadora de institutos nacionales correspondend de salud y hospitales de alta especialidad direccion general de coordinacion de los hospitales federales de referencia direccion de planeacion y desarrollo administrativo secretaria de salud salud of. n/dgchfr/rh/ /2012 113 mexico, d. f., a 12 de enero de 2012 asunto: referente a solicitud de informacion no. 0001200005312 lic. david e. waag director del centro de documentacion institucional y servidor publico habilitado de la unidad de enlace donceles no. 39, colonia centro delegacion cuauhtemoc, c. p. 06010 presente con el proposito de dar cumplimiento a los articulos 1, 2, 7, 28 fraccion viii, 41 y demas relativos a la ley federal de transparencia y acceso a la informacion publica gubernamental (ifai) y en respuesta a la solicitud no. 0001200005312 del oficio ue/18/2012 recibido el 04 de enero de 2012, el cual a continuacion se transcribe: con fundamento en lo previsto en los articulos 4, 7 (parrafo u

En el caso de arriba podemos ver que la estamos clasificando (flag_rechazo) como que la respuesta ha sido rechazada dado que encontramos la frase "informacion confidencial", se revisaron varios casos y se encontró la frase "no esta abarcando ninguna informacion confidencial" la cual no debería de considerarse como un rechazo, crearemos una variable auxiliar para ver en cuantos casos encontramos esta frase y poder discriminar de una manera mas efectiva.

Probamos primero con un subset para ver que todo funcione correctamente.

In [34]:
validacion_texto_aux = ['no esta abarcando ninguna informacion confidencial']

In [35]:
df_subset['flag_rechazo_aux'] = df_subset['texto_respuesta_adjunto'].apply(lambda x: any([k in x for k in validacion_texto_aux]))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
df_subset.head()

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto,respuestareal,flag_rechazo,flag_rechazo_aux
22,0001200005312,2012-01-01 14:15:43,secretaria de salud,terminada,electronica,informacion publica,con fundamento en previsto en los articulos 4 ...,por favor no referenciar a compranet; la prese...,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,favor de ver el archivo anexo.,https://www.infomex.org.mx/gobiernofederal/mod...,2012-01-17,mexico,mexico,naucalpan de juarez,53840,salud,pdf,1.0,20t secre 16 ene 2012 comision coordinadora de...,entrega de informacion en medio electronico,True,True
23,0001200005412,2012-01-01 14:18:31,secretaria de salud,terminada,electronica,informacion publica,con fundamento en previsto en los articulos 4 ...,por favor no referenciar a compranet; la prese...,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,favor de ver el archivo anexo.,https://www.infomex.org.mx/gobiernofederal/mod...,2012-01-16,mexico,mexico,naucalpan de juarez,53840,salud,pdf,1.0,39 idos comision coordinadora de institutos na...,entrega de informacion en medio electronico,True,True
24,0001200005512,2012-01-01 14:21:28,secretaria de salud,terminada,electronica,informacion publica,con fundamento en previsto en los articulos 4 ...,por favor no referenciar a compranet; la prese...,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,favor de ver el archivo anexo.,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,mexico,naucalpan de juarez,53840,salud,pdf,1.0,0492 comision coordinadora de institutos nacio...,entrega de informacion en medio electronico,True,True
25,0001200005612,2012-01-01 14:24:43,secretaria de salud,terminada,electronica,informacion publica,con fundamento en previsto en los articulos 4 ...,por favor no referenciar a compranet; la prese...,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,favor de ver el archivo anexo.,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,mexico,naucalpan de juarez,53840,salud,pdf,1.0,4h comision coordinadora de institutos naciona...,entrega de informacion en medio electronico,True,True
26,0001200005712,2012-01-01 14:28:28,secretaria de salud,terminada,electronica,informacion publica,con fundamento en previsto en los articulos 4 ...,por favor no referenciar a compranet; la prese...,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,favor de ver el archivo anexo.,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,mexico,naucalpan de juarez,53840,salud,pdf,1.0,$89 comision coordinadora de institutos nacion...,entrega de informacion en medio electronico,True,False


Ahora se lo aplicaremos al dataframe original.

In [37]:
df['flag_rechazo_aux'] = df['texto_respuesta_adjunto'].apply(lambda x: any([k in x for k in validacion_texto_aux]))

In [38]:
df.head(30)

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto,respuestareal,flag_rechazo,flag_rechazo_aux
0,1857500001912,2012-01-01 05:04:07,pemex exploracion y produccion,terminada,electronica,informacion publica,copia de los documentos y los con que cuenta a...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 02...,entrega de informacion en medio electronico,False,False
1,1857500002012,2012-01-01 05:13:16,pemex exploracion y produccion,terminada,electronica,informacion publica,del informacion y usados en la de flujo de efe...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 01...,entrega de informacion en medio electronico,False,False
2,0001800000712,2012-01-01 09:46:39,secretaria de energia,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,inexistencia de la informacion solicitada,el asunto se turno a la direccion general de r...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,energia,zip,2.0,None,inexistencia de la informacion solicitada,False,False
3,0000600003712,2012-01-01 10:11:41,secretaria de hacienda y credito publico,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,la informacion esta disponible publicamente,direcci&oacute;n de internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,hacienda y credito publico,pdf,1.0,inidos secretaria de hacienda y credito public...,la informacion esta disponible publicamente,False,False
4,0001500002812,2012-01-01 10:32:15,secretaria de desarrollo agrario territorial y...,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,mexico,distrito federal,miguel hidalgo,11230,reforma agraria,zip,4.0,None,entrega de informacion en medio electronico,False,False
5,0001100003612,2012-01-01 10:35:43,secretaria de educacion publica,terminada,electronica,informacion publica,quiero saber si el colegio abraham castellanos...,esta ubicada en calle 5ta. y mutualismo # 7685...,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-01-30,mexico,baja california,tijuana,22457,educacion publica,doc,1.0,None,entrega de informacion en medio electronico,False,False
6,0001600004112,2012-01-01 10:45:58,secretaria de medio ambiente y recursos naturales,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,se anexa archivo,https://www.infomex.org.mx/gobiernofederal/mod...,2012-01-31,mexico,distrito federal,miguel hidalgo,11230,medio ambiente y recursos naturales,zip,4.0,None,entrega de informacion en medio electronico,False,False
7,0416000000312,2012-01-01 11:21:46,consej

Crearemos el flag de rechazo definitivo en función de los dos flag que creamos.

In [39]:
def label_rechazo(row):
    aux = row['flag_rechazo']
    if row['flag_rechazo'] == True and row['flag_rechazo_aux'] == True:
        return False
    else:
        return aux

In [40]:
df['flag_rechazo_final'] = df.apply(lambda row: label_rechazo(row), axis=1)

In [41]:
df.head(30)

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto,respuestareal,flag_rechazo,flag_rechazo_aux,flag_rechazo_final
0,1857500001912,2012-01-01 05:04:07,pemex exploracion y produccion,terminada,electronica,informacion publica,copia de los documentos y los con que cuenta a...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 02...,entrega de informacion en medio electronico,False,False,False
1,1857500002012,2012-01-01 05:13:16,pemex exploracion y produccion,terminada,electronica,informacion publica,del informacion y usados en la de flujo de efe...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 01...,entrega de informacion en medio electronico,False,False,False
2,0001800000712,2012-01-01 09:46:39,secretaria de energia,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,inexistencia de la informacion solicitada,el asunto se turno a la direccion general de r...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,energia,zip,2.0,None,inexistencia de la informacion solicitada,False,False,False
3,0000600003712,2012-01-01 10:11:41,secretaria de hacienda y credito publico,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,la informacion esta disponible publicamente,direcci&oacute;n de internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,hacienda y credito publico,pdf,1.0,inidos secretaria de hacienda y credito public...,la informacion esta disponible publicamente,False,False,False
4,0001500002812,2012-01-01 10:32:15,secretaria de desarrollo agrario territorial y...,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,mexico,distrito federal,miguel hidalgo,11230,reforma agraria,zip,4.0,None,entrega de informacion en medio electronico,False,False,False
5,0001100003612,2012-01-01 10:35:43,secretaria de educacion publica,terminada,electronica,informacion publica,quiero saber si el colegio abraham castellanos...,esta ubicada en calle 5ta. y mutualismo # 7685...,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-01-30,mexico,baja california,tijuana,22457,educacion publica,doc,1.0,None,entrega de informacion en medio electronico,False,False,False
6,0001600004112,2012-01-01 10:45:58,secretaria de medio ambiente y recursos naturales,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,se anexa archivo,https://www.infomex.org.mx/gobiernofederal/mod...,2012-01-31,mexico,distrito federal,miguel hidalgo,11230,medio ambiente y recursos naturales,zip,4.0,None,entrega de informacion en medio electronico

In [42]:
filtro = df['flag_rechazo_final'] == True
df_subset_1 = df[filtro]

In [43]:
df_subset_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70729 entries, 26 to 1554538
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   folio                        70729 non-null  object        
 1   fechasolicitud               70729 non-null  datetime64[ns]
 2   dependencia                  70729 non-null  object        
 3   estatus                      70729 non-null  object        
 4   medioentrada                 70729 non-null  object        
 5   tiposolicitud                70729 non-null  object        
 6   descripcionsolicitud         70729 non-null  object        
 7   otrosdatos                   70729 non-null  object        
 8   archivoadjuntosolicitud      70729 non-null  object        
 9   medioentrega                 70729 non-null  object        
 10  fechalimite                  67921 non-null  datetime64[ns]
 11  respuesta                    70729 non

In [44]:
df_subset_1.head(20)

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto,respuestareal,flag_rechazo,flag_rechazo_aux,flag_rechazo_final
26,0001200005712,2012-01-01 14:28:28,secretaria de salud,terminada,electronica,informacion publica,con fundamento en previsto en los articulos 4 ...,por favor no referenciar a compranet; la prese...,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,favor de ver el archivo anexo.,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,mexico,naucalpan de juarez,53840,salud,pdf,1.0,$89 comision coordinadora de institutos nacion...,entrega de informacion en medio electronico,True,False,True
117,0000700003112,2012-01-01 23:13:22,secretaria de la defensa nacional,terminada,electronica,informacion publica,de elementos activos de la distribucion en reg...,distribucion total de efectivos de la sdn por ...,None,entrega por internet en el infomex,2012-02-02,negativa por ser reservada o confidencial,se anexa resolucion,https://www.infomex.org.mx/gobiernofederal/mod...,2012-01-15,mexico,mexico,tlalnepantla de baz,54080,defensa nacional,pdf,1.0,22332 secretaria de la defensa nacional estado...,negativa por ser reservada o confidencial,True,False,True
128,0001700002412,2012-01-02 08:50:36,procuraduria general de la republica,terminada,electronica,informacion publica,qusiera se me por escrito si sancion denuncia ...,al parecer esta impedido de ocupar cargos publ...,None,entrega por internet en el infomex,2012-03-02,la solicitud no corresponde al marco de la ley,la unidad de enlace no es el medio idoneo para...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,nayarit,tepic,63502,procuraduria general de la republica,PDF,1.0,"""subprocuraduria juridica y de asuntos ados y,...",la solicitud no corresponde al marco de la ley,True,False,True
136,0002200002112,2012-01-02 10:39:54,secretaria de seguridad publica (ahora incorpo...,terminada,electronica,informacion publica,1. diga si un procedimiento administrativo de ...,organo interno de control en la secretaria de ...,None,entrega por internet en el infomex,2012-03-02,inexistencia de la informacion solicitada,ver archivo adjunto,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,iztapalapa,9780,seguridad publica,pdf,1.0,secretaria de seguridad publica ssp unidad de ...,inexistencia de la informacion solicitada,True,False,True
143,0000800001612,2012-01-02 11:24:56,secretaria de agricultura ganaderia desarrollo...,terminada,electronica,informacion publica,solicito copia de los documentos presentados p...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,se anexa informacion,https://www.infomex.org.mx/gobiernofederal/mod...,2012-01-19,mexico,sonora,hermosillo,83190,"agricultura, ganaderia, desarrollo rural, pesc...",pdf,1.0,subsecretaria de agricultura oficio no 300.-01...,entrega de informacion en medio electronico,True,False,True
165,0000600004312,2012-01-02 12:47:36,secretaria de hacienda y credito publico,terminada,electronica,informacion publica,solicito tabulador regional en certificadas de...,solicito tabulador regional en dos copias cert...,None,copia certificada,2012-03-02,inexistencia de la informacion solicitada,resolucion c.i. 057/2012 del comite de informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3103,hacienda y credito publico,pdf,1.0,secretaria de hacienda y credito publico comit...,inexistencia de la informacion solicitada,True,False,True
230,1857200003012,2012-01-02 15:44:37,petroleos mexicanos,terminada,electronica,informacion publica,solicito de la empresa petroleos proporcione l...,Non

In [45]:
df_subset_1['flag_rechazo_aux'].unique()

array([False])

Validamos algunos casos al azar para ver que tan consistente es la forma de discriminar dadas las frases que se definieron que implicaban un rechazo.

In [46]:
df_subset_1['texto_respuesta_adjunto'].loc[269]

'conacyt anos inidos direccion adjunta de asuntos juridicos unidad de enlace ciudad de mexico, el 24 de enero de 2012. estimado (a) solicitante: referente a la solicitud de acceso a la informacion con numero de folio 1111200000712, de fecha 05 de enero de 2012, donde requiere lo siguiente: [...] *numero de becas vigentes (bajo las convocatorias becas nacionales conacyt) otorgadas a extranjeros para realizar estudios de posgrado en mexico. **porcentaje que representan las becas otorgadas a extranjeros sobre el total de becas asignadas vigentes. ***nacionalidad de los extranjeros que representan el mayor porcentaje de beneficiados con estas becas. [...] ahora bien, la solicitud fue turnada el mismo dia de su recepcion a traves de correo electronico a la direccion adjunta de posgrado y becas,-er lo sucesivo conacyt-, por tratarse de un asunto de su competencia. en fecha 24 de enero de 2012 a traves de correo electronico, el lic. jose cuellar helox, servidor publico habilitado de la direcc

### Aplicamos la función con las reglas de negocio para actualizar la columna respuestareal

In [47]:
df['respuestareal'] = df.apply(lambda x: fe.RespuestaReal(x['respuesta'], x['tipo_archivo_respuesta'], x['cantidad_archivos_respuesta'], x['flag_rechazo_final']), axis=1)

In [48]:
df = df.drop(columns=['flag_rechazo', 'flag_rechazo_aux', 'flag_rechazo_final'])

In [49]:
df.head(30)

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto,respuestareal
0,1857500001912,2012-01-01 05:04:07,pemex exploracion y produccion,terminada,electronica,informacion publica,copia de los documentos y los con que cuenta a...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 02...,entrega de informacion en medio electronico
1,1857500002012,2012-01-01 05:13:16,pemex exploracion y produccion,terminada,electronica,informacion publica,del informacion y usados en la de flujo de efe...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 01...,entrega de informacion en medio electronico
2,0001800000712,2012-01-01 09:46:39,secretaria de energia,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,inexistencia de la informacion solicitada,el asunto se turno a la direccion general de r...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,energia,zip,2.0,None,inexistencia de la informacion solicitada
3,0000600003712,2012-01-01 10:11:41,secretaria de hacienda y credito publico,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,la informacion esta disponible publicamente,direcci&oacute;n de internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,hacienda y credito publico,pdf,1.0,inidos secretaria de hacienda y credito public...,la informacion esta disponible publicamente
4,0001500002812,2012-01-01 10:32:15,secretaria de desarrollo agrario territorial y...,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,mexico,distrito federal,miguel hidalgo,11230,reforma agraria,zip,4.0,None,sin respuesta
5,0001100003612,2012-01-01 10:35:43,secretaria de educacion publica,terminada,electronica,informacion publica,quiero saber si el colegio abraham castellanos...,esta ubicada en calle 5ta. y mutualismo # 7685...,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-01-30,mexico,baja california,tijuana,22457,educacion publica,doc,1.0,None,entrega de informacion en medio electronico
6,0001600004112,2012-01-01 10:45:58,secretaria de medio ambiente y recursos naturales,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,se anexa archivo,https://www.infomex.org.mx/gobiernofederal/mod...,2012-01-31,mexico,distrito federal,miguel hidalgo,11230,medio ambiente y recursos naturales,zip,4.0,None,sin respuesta
7,0416000000312,2012-01-01 11:21:46,consejo nacional de poblacion,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,en

## Guardamos en la S3 en la carpeta mlpreproc

In [50]:
s3_url = f's3://inai-summerofdata/mlpreproc/inai.parquet'
formatos = {'folio':str, 'fechasolicitud':'datetime64', 'dependencia':str,  'estatus':str,
           'medioentrada':str, 'tiposolicitud':str, 'descripcionsolicitud':str, 'otrosdatos':str,
           'archivoadjuntosolicitud':str, 'medioentrega':str, 'fechalimite':'datetime64',
           'respuesta':str, 'textorespuesta':str, 'archivorespuesta':str, 'fecharespuesta':'datetime64',
           'pais':str, 'estado':str, 'municipio':str, 'codigopostal':str, 'sector':str,
           'tipo_archivo_respuesta':str, 'cantidad_archivos_respuesta':'float64', 
            'texto_respuesta_adjunto':str, 'respuestareal':str}
df.astype(formatos).to_parquet(s3_url, engine='auto', compression='snappy')